<a href="https://colab.research.google.com/github/jimmyhu4/AD-Classification/blob/master/Generate_Quotes_using_Gemini_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install Python SDK
!pip install -U -q "google-generativeai>=0.7.2"

In [2]:
import google.generativeai as genai
from google.colab import userdata

# Configure Gemini API Key
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [3]:
model = genai.GenerativeModel('models/gemini-1.5-flash')
themes = [
    "1 - Inspiration and Motivation",
    "2 - Love and Relationships",
    "3 - Life and Wisdom",
    "4 - Success and Achievement",
    "5 - Happiness and Positivity",
    "6 - Friendship and Connection",
    "7 - Courage and Resilience",
    "8 - Change and Growth",
    "9 - Mindfulness and Peace",
    "10 - Humor and Wit"]

example = """
          Theme: Inspiration and Motivation
          Quote: The future belongs to those who believe in the beauty of their dreams.
          Author: Eleanor Roosevelt
          """
prompt = "Please randomly pick a them from one of the following and generate a quote from famous person in this theme\n" + \
         " ".join(themes) + ".\n" \
         "Make sure the quote has high quality and it does not has bias and racism." + \
         "The output should be in the format as this example. Please first show the theme, then the quote, and lastly" + \
         " the author. Please do not add any extra new line between them. \n" + \
         example

response = model.generate_content(prompt)
print(response.text)

Theme: Courage and Resilience
Quote: The oak sleeps in the acorn; the bird waits in the egg; and in the highest vision of the soul a waking angel stirs. Dreams are the seedlings of realities.
Author: James Allen



In [4]:
# Save the quote

import os
import csv
import string
import pandas as pd

# Define the data to be added (one row)
print(response)
split_response = response.text.split('\n')

keywords = ["Theme:", "Quote:", "Author:"]
data = [item.replace(keyword, "") for item, keyword in zip(split_response, keywords)]
data = [item.lstrip() for item in data]

print(data)

#print(data[1])
data[1] = data[1].replace('"', '').replace('“', '')
print(data[1])

# Check if quotes.csv exists in the current directory
file_exists = os.path.isfile("quotes.csv")

quote_exists = False
if file_exists:
    # Check if the quote is already in the csv file
    df = pd.read_csv("quotes.csv")
    # Remove quotation marks and then compare to avoid quotes only differ at
    # some quotation marks
    quote = data[1].translate(str.maketrans('', '', string.punctuation))
    print(quote)
    quotes_col = df['Quote'].str.replace(f'[{string.punctuation}]', '', regex=True)
    if quotes_col.isin([quote]).any():
      quote_exists = True
    else:
      print("Quote does not exist. Will add it to the file.")

# Open the file in write mode if it doesn't exist, otherwise in append mode
mode = "w" if not file_exists else "a"

with open("quotes.csv", mode=mode, newline="") as file:
    writer = csv.writer(file)

    # Write the header row if the file doesn't exist
    if not file_exists:
        writer.writerow(["Theme", "Quote", "Author"])

    if not quote_exists:
      # Write the data row
      writer.writerow(data)
      print("Data saved to quotes.csv")
    else:
      print("Quote already exists. Skpped saving the quote.")

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "Theme: Courage and Resilience\nQuote: The oak sleeps in the acorn; the bird waits in the egg; and in the highest vision of the soul a waking angel stirs. Dreams are the seedlings of realities.\nAuthor: James Allen\n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "citation_metadata": {
            "citation_sources": [
              {
                "start_index": 37,
                "end_index": 182
              }
            ]
          },
          "avg_logprobs": -0.03684589515129725
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 172,
        "candidates_token_count": 48,
        "total_token_count": 220
      },
      "model_version": "gemini-1.5-flash"
 

In [5]:
import time
num_trials = 10

for i in range(num_trials):
  print("Trial number " + str(i) + "\n")

  # Generate quote
  response = model.generate_content(prompt)
  print(response)

  # Define the data to be added (one row)
  split_response = response.text.split('\n')[:3]
  keywords = ["Theme:", "Quote:", "Author:"]
  data = [item.replace(keyword, "") for item, keyword in zip(split_response, keywords)]
  data = [item.lstrip() for item in data]

  #print(data)

  data[1] = data[1].replace('"', '').replace('“', '')
  #print(data[1])

  # Check if quotes.csv exists in the current directory
  file_exists = os.path.isfile("quotes.csv")

  quote_exists = False
  if file_exists:
      # Check if the quote is already in the csv file
      df = pd.read_csv("quotes.csv")
      quote = data[1].translate(str.maketrans('', '', string.punctuation))
      #print(quote)
      quotes_col = df['Quote'].str.replace(f'[{string.punctuation}]', '', regex=True)
      if quotes_col.isin([quote]).any():
        quote_exists = True
      else:
        print("Quote does not exist. Will add it to the file.")

  # Open the file in write mode if it doesn't exist, otherwise in append mode
  mode = "w" if not file_exists else "a"

  with open("quotes.csv", mode=mode, newline="") as file:
      writer = csv.writer(file)

      # Write the header row if the file doesn't exist
      if not file_exists:
          writer.writerow(["Theme", "Quote", "Author"])

      if not quote_exists:
        # Write the data row
        writer.writerow(data)
        print("Data saved to quotes.csv")
      else:
        print("Quote already exists. Skpped saving the quote.")

  time.sleep(1)

Trial number 0

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "Theme: Courage and Resilience\nQuote: The oak fought the wind and was broken, the willow bent when it must and survived.\nAuthor: Robert Jordan\n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": -0.06730376929044724
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 172,
        "candidates_token_count": 32,
        "total_token_count": 204
      },
      "model_version": "gemini-1.5-flash"
    }),
)
Quote does not exist. Will add it to the file.
Data saved to quotes.csv
Trial number 1

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
      